In [ ]:
import re
import gurobipy
import cobra
from cobra.flux_analysis.gapfilling import GapFiller
from cobra.io import read_sbml_model, write_sbml_model
from cobra import exceptions
import sys, os

In [ ]:
def fake_copy(model):
    write_sbml_model(model, filename="tmp.xml")
    copied_model = read_sbml_model("tmp.xml")
    os.remove("tmp.xml")
    return copied_model

In [ ]:
# Using fake_copy()
# "model" variable changed by "Model"
def load_query_model(model, obj = None):
    """
    Loads a model. Objective can be changed to either "biomass" (regex
    will be used) or a valid reaction ID.
    Print is used temporary. A general log will be generated in the 
    final version.
    """
    Model = fake_copy(model)
    if obj == "biomass":
        b = re.compile("biomass", re.IGNORECASE)
        bc = re.compile("(biomass){1}.*(core){1}", re.IGNORECASE)
        reactions = [reaction.id for reaction in Model.reactions]
        # searching for a biomass core reaction
        core = list(filter(bc.match, reactions))
        if core:
            Model.objective = core[0]
            return Model
        # searching for a non core biomass reaction
        biomass = list(filter(b.match, reactions))
        if biomass:
            Model.objective = biomass[0]
            return model
        print("biomass not found in the query model")
        return Model
    if obj != "biomass" and obj != None:
        try:
            Model.objective = obj
        except ValueError:
            print(str(obj) + " not found in the query model")
            return Model
        return Model
    if obj == None:
        return Model

In [ ]:
# Using fake_copy()
def load_template_models(template_list, obj = None):
    """
    Takes a list of template models and changes objective if specified.
    Objective can be either "biomass" or a specific reaction ID.
    """
    templates = []
    for t in template_list:
        templates.append(fake_copy(t))
    failures = []
    if obj == None:
        return templates
    if obj == "biomass":
        b = re.compile("biomass", re.IGNORECASE)
        bc = re.compile("(biomass){1}.*(core){1}", re.IGNORECASE) 
        for template in templates:
            reactions = [reaction.id for reaction in template.reactions]
            # Searching for a biomass_core reaction
            core = list(filter(bc.match, reactions))
            if core:
                template.objective = core[0]
                continue
            # Searching for a non core biomass reaction
            biomass = list(filter(b.match, reactions))
            if biomass:
                template.objective = biomass[0]
            # If biomass reactions are not found, the model name is stored into "failures" list.
            # The model won't change its objective, but it will be used anyway for gap filling
            else:
                failures.append(str(template))
            print("Objective not changed for:\n", failures)
            return templates
    if obj != None and obj != "biomass":
        for template in templates:
            try:
                template.objective = obj
            except ValueError:
                failures.append(str(template))
        print("Objective not changed for:\n", failures)
        return templates

In [ ]:
def gapfilling(model, template, integer_threshold, it = 1):
    """
    Calls GapFiller class from cobrapy.
    """
    gapfiller = GapFiller(model, template, integer_threshold = integer_threshold,
                         demand_reactions = False)
    return gapfiller.fill(iterations = it)

In [7]:
def add_exchange_reactions(model, template):
    """
    Adds all exchange reactions from a template matching model metabolites. 
    """
    EX = [template.reactions.get_by_id(i.id) for i in template.reactions if i.id.startswith("EX_")]
    for reaction in EX:
        if reaction not in model.reactions and str(list(reaction.metabolites.keys())[0]) in [i.id for i in model.metabolites]:
            model.add_reaction(reaction.copy())
    return model

In [9]:
def is_transport(reaction, compartments_list, all_compounds = False, ignore_h = False):
    """
    Takes a cobra model reaction as input and determines if it is a transport reaction.
    all_compounds = True -> both sides of the reaction must be the same
    all_compounds = False -> at least one compound must be in both sides
    ignore_h = True -> does not consider proton transference as transport
    """
    # left part of the reaction
    r = list(str(x) for x in reaction.reactants)
    # right part
    p = list(str(x) for x in reaction.products)
    if len(r) != len(p):
        return False
    c = compartments_list
    # removing terminations
    R = []
    for i in r:
        for e in c:
            i = i.replace(e, "") 
        R.append(i)
    P = []
    for i in p:
        for e in c:
            i = i.replace(e, "")
        P.append(i)
    # we sort the lists to avoid missing transport reactions where the sequence of the compounds is not maintained
    R = sorted(R)
    P = sorted(P)
    if all_compounds == False:
        if ignore_h == True:
            if "h" in R:
                R.remove("h")
            if "h" in P:
                P.remove("h")
        for i in R:
            if i in P:
                return True
            else:
                return False
    if R == P:
        return True
    else:
        return False

In [10]:
def add_transport(model, template, all_compounds = False, ignore_h = False):
    """
    Adds transport reactions from a template which metabolites are present in the model.
    """
    # PREPARATION
    # template compartments
    t_compartments = list(template.compartments.keys())
    t_Compartments = []
    for i in t_compartments:
        t_Compartments.append("_" + str(i))
    # getting compartments and metabolites from query model for further use
    m_compartments = list(model.compartments.keys())
    m_Compartments = []
    for i in m_compartments:
        m_Compartments.append("_" + str(i))
    m_metabolites = list(str(x) for x in model.metabolites)
    # removing suffixes from metabolites
    m_Metabolites = []
    for i in m_metabolites:
        for e in m_Compartments:
            i = i.replace(e, "")
        m_Metabolites.append(i)
    # sorting and removing duplicates
    m_Metabolites = list(dict.fromkeys(m_Metabolites))
    # REACTION ADDING
    for reaction in template.reactions:
        if is_transport(reaction, t_Compartments, all_compounds = all_compounds, ignore_h = ignore_h) and reaction not in model.reactions:
            # we will only use reactants as they'll be the same as products (apart from location)
            m = list(str(x) for x in reaction.reactants)
            M = []
            for i in m:
                for e in t_Compartments:
                    i = i.replace(e, "")
                M.append(i)
            if all(x in m_Metabolites for x in M):  
                model.add_reaction(reaction.copy())
    return model


In [30]:
# Changing "model" variable by "Model" and "templates" by "Templates"
def homology_gapfilling(model, templates, model_obj = None, template_obj = None, use_all_templates = False,
                       integer_threshold = 1e-6, force_exchange = False, force_transport = False, t_all_compounds = False,
                       t_ignore_h = False, value_fraction = 0.8):
    """
    Performs gap filling on a model using homology models as templates.
    """
    Model = load_query_model(model, obj = model_obj)
    Model.solver = 'gurobi'
    Templates = load_template_models(templates, obj = template_obj)
    # this dict will store used models, genes and reactions
    added_reactions = {}
    # initial flux value
    value = Model.optimize().objective_value
    if value == None:
        value = 0.0
    if use_all_templates == False:
        for template in Templates:
            # adding exchange reactions
            if force_exchange == True:
                add_exchange_reactions(Model, template)
            # adding transport reactions
            if force_transport == True:
                add_transport(Model, template, all_compounds = t_all_compounds, ignore_h = t_ignore_h)
            template.solver = 'gurobi'
            # reactions "log"
            log = []
            try:
                # result variable will store the reactions ids
                result = gapfilling(Model, template, integer_threshold = integer_threshold)
                for reaction in result[0]:
                    if reaction.id.startswith("EX_"):
                        log.append((reaction.id, "Exchange reaction"))
                    else:
                        log.append((reaction.id, [str(list(reaction.genes)[i]) 
                                              for i in range(len(list(reaction.genes)))]))
                added_reactions[str(template)] = log
                # Adding reactions to the model
                [Model.add_reaction(reaction.copy()) for reaction in result[0]]
                # Flux will be evaluated here
                new_value = Model.optimize().objective_value
                if new_value != None and new_value > value:
                    value = new_value
                elif new_value == None:
                    continue
                elif new_value != None and new_value == value:
                    break
                elif new_value < value:
                    if new_value >= value * value_fraction:
                        for i in range(len(log)):
                            Model.reactions.log[i][0].lower_bound = 0.
                            Model.reactions.log[i][0].upper_bound = 0.
                        new_value = Model.optimize().objective_value
                        value = new_value
                    else:
                        [Model.remove_reactions(Model.reactions.log[i][0]) for i in range(len(log))]
                        del added_reactions[str(template)]
                        break
                    
            except RuntimeError:
                print("\n" + str(template) + ": failed to validate gapfilled model, try lowering the integer_threshold")
            except exceptions.Infeasible:
                print("\n" + str(template) + ": gapfilling optimization failed (infeasible)")
        return Model, added_reactions
    else:
        for template in Templates:
            # adding exchange reactions
            if force_exchange == True:
                add_exchange_reactions(Model, template)
            # adding transport reactions
            if force_transport == True:
                add_transport(Model, template, all_compounds = t_all_compounds, ignore_h = t_ignore_h)
            template.solver = 'gurobi'
            log = []
            try:
                result = gapfilling(Model, template, integer_threshold = integer_threshold)   
                for reaction in result[0]:
                    if reaction.id.startswith("EX_"):
                        log.append((reaction.id, "Exchange reaction"))
                    else:
                        log.append((reaction.id, [str(list(reaction.genes)[i]) 
                                              for i in range(len(list(reaction.genes)))]))
                added_reactions[str(template)] = log
                [Model.add_reaction(reaction.copy()) for reaction in result[0]]
            except RuntimeError:
                print("\n" + str(template) + ": failed to validate gapfilled model, try lowering the integer_threshold")
            except exceptions.Infeasible:
                print("\n" + str(template) + ": gapfilling optimization failed (infeasible)") 
        return Model, added_reactions

In [1]:
iEC1344_C = read_sbml_model("iEC1344_C.xml")
iEC1364_W = read_sbml_model("iEC1364_W.xml")
iJN746 = read_sbml_model("iJN746.xml")

NameError: name 'read_sbml_model' is not defined

In [27]:
x = load_query_model(iEC1364_W)

In [31]:
T = [iJN746, iEC1344_C]

In [32]:
X, added_reactions = homology_gapfilling(x, T, use_all_templates=True)

Read LP format model from file /tmp/tmpffcswcgi.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmpueim988_.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmpgf0emk1i.lp
Reading time = 0.01 seconds
: 907 rows, 2108 columns, 8802 nonzeros
Read LP format model from file /tmp/tmps1afps23.lp
Reading time = 0.00 seconds
: 907 rows, 2108 columns, 8802 nonzeros
Read LP format model from file /tmp/tmpngk1791c.lp
Reading time = 0.01 seconds
: 1927 rows, 5530 columns, 21806 nonzeros
Read LP format model from file /tmp/tmpat3jv5lk.lp
Reading time = 0.01 seconds
: 1927 rows, 5530 columns, 21806 nonzeros
Read LP format model from file /tmp/tmpwogcdd_2.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmp6gjjjdew.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros


Ignoring reaction 'BIOMASS_KT_TEMP' since it already exists.
Ignoring reaction 'BIOMASS_KT_TEMP' since it already exists.


In [33]:
added_reactions

{'iJN746': [('BIOMASS_KT_TEMP', [])], 'iEC1344_C': [('BIOMASS_KT_TEMP', [])]}

In [34]:
X.optimize().objective_value

0.18761640097381724